In [85]:
from __future__ import print_function
from collections import defaultdict
from IPython.display import display, clear_output
import sys
import utils
from copy import deepcopy
from utils import accepts_tuple_arg, add_tuples
def scrollUp(n=1):
    for i in range(0,n):
        sys.stdout.write('\x1b[1A')
        sys.stdout.flush()

def scrollDown(n=1):
    for i in range(0,n):
        sys.stdout.write('\n')
        sys.stdout.flush()
from time import sleep

POSSIBLE_MOVES = [(-1,0),(1,0),(0,-1),(0,1)]
COLORS = [utils.redBG, utils.greenBG,utils.yellowBG, utils.blueBG, utils.magentaBG, utils.cyanBG, utils.darkGrayBG, utils.whiteBG]
class State:
    def __init__(self, filepath, moveDelay = .5):
        self.firstPrint = True #Utility
        self.moveDelay = moveDelay
        self.numExpanded = 0
        self.grid = []
        self.endPoints = defaultdict(list)        
        self.visited = defaultdict(lambda: defaultdict(bool))
        textFile = open(filepath)
        lines = textFile.readlines()
        for i, line in enumerate(lines):
            for j, x in enumerate(line):
                if x == '_':
                    continue
                self.endPoints[x].append((j,i))
                
            line = list(line)
            self.grid.append(line)
        self.history = [self.grid]
        self.dimensions = len(self.grid[0])-1 # don't count newline
    @accepts_tuple_arg
    def move(self, x, y, targets):
        #assert(self.shortestPaths.get((x,y)) != None)
        targets = list(targets)
        try: 
            targets.remove((x,y))
        except ValueError:
            pass

        self.targets = targets
        key = tuple(sorted(targets))
        
        self.visited[(x,y)][key] = True
        self.location = (x,y)
        self.currentPath = self.shortestPaths[(x,y)][key]
        legalMoves = self.getTransitions()
        for move in legalMoves:
            #If we found a shorter path than the current shortest, swap it out.
            self.updateShortestPath(move[0], targets, self.currentPath+[self.location])
        self.numExpanded +=1
        if (self.moveDelay == 0):
            return
        sleep(self.moveDelay)
        self.printStatus()

###########################
############ UTILITIES ####
###########################
    #returns a copy of the grid, with optional paths filled in
    def getGridCopy(self):
        return deepcopy(self.grid)
    def getColors(self):
        return list(self.endPoints.keys())

    def __str__(self):
        return ''.join([''.join(row) for row in self.grid])
    
    def colorize(self):
        string = str(self)
        string = string.replace('_', utils.lightGrayBG(" "))
        keys = list(self.endPoints.keys())
        for i, color in enumerate(keys):
            string = string.replace(color, COLORS[i](color))
        return string
    
    def printStatus(self):
        maze_height = len(self.grid)
        if self.firstPrint:
            self.firstPrint = False
        else:
            scrollUp(maze_height+3)
        #sys.stdout.write(self.colorize()+"\nCurrent Position: \nNodes expanded: \nPath Length: \n")
        #sys.stdout.flush()
        print(self.colorize()+"\nCurrent Position: \nNodes expanded: \nPath Length: \n")
    @accepts_tuple_arg
    def getCoord(self, x, y):
        if x < 0 or y < 0 or x >= self.dimensions or y >= self.dimensions:
            return None
        return self.grid[y][x]
    @accepts_tuple_arg
    def setCoord(self, x, y, val):
        self.grid[y][x] = str(val)
    @accepts_tuple_arg    
    def isWall(self, x, y):
        return self.getCoord(x,y) == '%'
def gridToString(grid, colors):
    string = ''.join([''.join(row) for row in grid])
    string = string.replace('_', utils.lightGrayBG(" "))
    for i, color in enumerate(colors):
        string = string.replace(color, COLORS[i](color))
    return string
@accepts_tuple_arg
def getCoord(grid, x, y):
    if x < 0 or y < 0 or x >= (len(grid[0])-1) or y >= (len(grid[0])-1):
        return None
    return grid[y][x]
@accepts_tuple_arg
def setCoord(grid, x, y, val, overwrite=True):
    if not overwrite and grid[y][x] != "_":
        return
    grid[y][x] = str(val)

def fillPath(grid, color, path, makeCopy = True):
    if makeCopy:
        grid = deepcopy(grid)
    for step in path:
        setCoord(grid, step, color, False)
    return grid
@accepts_tuple_arg
def getNeighborCoords(grid, x, y):
    return [add_tuples((x,y),move) for move in POSSIBLE_MOVES]
@accepts_tuple_arg
def getEmptyNeighborCoords(grid, x, y):
    neighbors = getNeighborCoords(grid, x, y)
    return [neighbor for neighbor in neighbors if getCoord(grid,neighbor)=="_" ]
@accepts_tuple_arg
def countNeighborsWithColor(grid, color, x, y):
    neighbors = [add_tuples((x,y), move) for move in POSSIBLE_MOVES]
    neighbors = [1 if getCoord(grid,neighbor)==color else 0 for neighbor in neighbors ]
    return sum(neighbors)
if __name__ == "__main__":
    m1 = State("free_flow/input55.txt")    
    m1.printStatus()
    sleep(.5)
    m1.printStatus()

B  RO
   Y 
  Y  
 RO G
 BG  
Current Position: 
Nodes expanded: 
Path Length: 

B  RO
   Y 
  Y  
 RO G
 BG  
Current Position: 
Nodes expanded: 
Path Length: 



In [86]:
m1 = State("free_flow/input55.txt")
print(m1.endPoints)
colors = m1.getColors()

defaultdict(<class 'list'>, {'Y': [(3, 1), (2, 2)], 'O': [(4, 0), (2, 3)], 'R': [(3, 0), (1, 3)], 'B': [(0, 0), (1, 4)], '\n': [(5, 0), (5, 1), (5, 2), (5, 3)], 'G': [(4, 3), (2, 4)]})


In [87]:
def satisfiesConstraints(grid, color, path):
    grid = fillPath(grid, color, path)
    constraints = [noOverlapConstraint, noZigzagConstraint]
    for move in path:
        if not all([constraint(grid, color, move) for constraint in constraints]):
            return False
    return True
#Fill path uses a modified setCoord that won't overwrite existing values
def noOverlapConstraint(grid, color, move):
    return getCoord(grid, move) == color
def noZigzagConstraint(grid, color, move):
    return countNeighborsWithColor(grid, color, move) <=2

In [88]:
"""
Frontier is a list of paths
"""

def findAllPaths(state, color):
    start, end = state.endPoints[color]
    paths = defaultdict(bool)
    frontier = [[start]]
    finishedPaths = []
    paths[tuple([start])] = True
    while len(frontier) > 0:
        currPath = frontier.pop()
        grid = fillPath(state.grid, color, currPath)
        neighbors = getEmptyNeighborCoords(grid, currPath[-1])
        for neighbor in neighbors:
            candidatePath = currPath + [neighbor]
            if end in getNeighborCoords(neighbor):
                finishedPaths+= candidatePath
                continue
            if satisfiesConstraints(state.grid, color, candidatePath):
                frontier.append(candidatePath)
    return finishedPaths
findAllPaths(m1, "B")
    

TypeError: getNeighborCoords() missing 1 required positional argument: 'y'